In [1]:
import pandas as pd
import json
import os
import re
import requests
import glob

In [24]:
rs_parks = pd.read_csv('data/retrograde_park_codes.csv')

In [27]:
pd.set_option('max.rows', 3000)
br_stadiums = pd.read_csv('data/complete_stadiums.csv', index_col = [0])
br_stadiums

,team_code,team,state_code,year,primary_stadium,primary_latitude,primary_longitude,secondary_stadium,secondary_latitude,secondary_longitude,attendance/game,pitching_park_factor,batting_park_factor
0,TBD,Tampa Bay Rays,FL,2019,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"14,552",96,97
1,TBD,Tampa Bay Rays,FL,2018,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"14,259",96,97
2,TBD,Tampa Bay Rays,FL,2017,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"15,477",96,96
3,TBD,Tampa Bay Rays,FL,2016,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"15,879",95,95
4,TBD,Tampa Bay Rays,FL,2015,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"15,322",97,96
5,TBD,Tampa Bay Rays,FL,2014,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"17,858",97,97
6,TBD,Tampa Bay Rays,FL,2013,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"18,646",96,97
7,TBD,Tampa Bay Rays,FL,2012,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"19,255",94,95
8,TBD,Tampa Bay Rays,FL,2011,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"18,879",92,93
9,TBD,Tampa Bay Rays,FL,2010,Tropicana Field,27.768333,82.653333,NaN,NaN,NaN,"23,025",94,95


In [6]:
br_unique = br_stadiums.primary_stadium.unique()
coordinates_list = []
for stadium in br_unique:
    dict_iter = {'stadium_name' : stadium,
                'lat' : None,
                'lon' : None,
                'state_code' : None}
    dict_iter['lat'] = br_stadiums[br_stadiums.primary_stadium == stadium].primary_latitude.value_counts().idxmax()
    dict_iter['lon'] = br_stadiums[br_stadiums.primary_stadium == stadium].primary_longitude.value_counts().idxmax()
    dict_iter['state_code'] = br_stadiums[br_stadiums.primary_stadium == stadium].state_code.value_counts().idxmax()
    coordinates_list.append(dict_iter)

In [7]:
unique_stadiums = pd.DataFrame(coordinates_list)

In [9]:
rs_parks.head()

,PARKID,NAME,AKA,CITY,STATE,START,END,LEAGUE,NOTES
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [19]:
all_park_ids = []
for j in range(len(rs_parks)):
    park_id = rs_parks.iloc[j]['PARKID']
    stadiums = [rs_parks.iloc[j]['NAME']]
    state_code = rs_parks.iloc[j]['STATE']
    if type(rs_parks.iloc[j]['AKA']) == str and ';' in rs_parks.iloc[j]['AKA']:
        stadiums += rs_parks.iloc[j]['AKA'].split(';')
    elif type(rs_parks.iloc[j]['AKA']) == str and ';' not in rs_parks.iloc[j]['AKA']:
        stadiums.append(rs_parks.iloc[j]['AKA'])
    else:
        pass
    for k in range(len(stadiums)):
        dict_iter = {'state_code' : state_code,
                    'stadium_name' : stadiums[k],
                    'park_id' : park_id}
        all_park_ids.append(dict_iter)
    

In [21]:
all_parks = pd.DataFrame(all_park_ids)

In [29]:
all_parks

,state_code,stadium_name,park_id
0,NY,Riverside Park,ALB01
1,PA,Columbia Park,ALT01
2,CA,Angel Stadium of Anaheim,ANA01
3,CA,Edison Field,ANA01
4,CA,Anaheim Stadium,ANA01
5,TX,Arlington Stadium,ARL01
6,TX,Rangers Ballpark in Arlington,ARL02
7,TX,The Ballpark in Arlington,ARL02
8,TX,Ameriquest Fl,ARL02
9,GA,Atlanta-Fulton County Stadium,ATL01


In [23]:
full_frame = br_stadiums.merge(all_parks, how = 'left', left_on = ['primary_stadium', 'state_code'],
                              right_on = ['stadium_name', 'state_code'])

In [25]:
full_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2590 entries, 0 to 2589
Data columns (total 15 columns):
team_code               2590 non-null object
team                    2590 non-null object
state_code              2590 non-null object
year                    2590 non-null int64
primary_stadium         2590 non-null object
primary_latitude        2590 non-null float64
primary_longitude       2590 non-null float64
secondary_stadium       82 non-null object
secondary_latitude      82 non-null object
secondary_longitude     82 non-null object
attendance/game         2590 non-null object
pitching_park_factor    2590 non-null int64
batting_park_factor     2590 non-null int64
stadium_name            2195 non-null object
park_id                 2195 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 323.8+ KB


In [27]:
problem_stadiums = full_frame[full_frame.park_id.isnull()]
problem_stadiums.primary_stadium.value_counts()

Griffith Stadium                51
Anaheim Stadium                 32
Stade Olympique                 28
Briggs Stadium                  23
Great American Ball Park        17
SkyDome                         16
Jack Murphy Stadium             16
Petco Park                      16
Rogers Centre                   15
U.S. Cellular Field             14
AT&T; Park                      13
Exhibition Stadium              12
Comiskey Park II                12
Nationals Park                  12
San Diego Stadium               12
Yankee Stadium III              11
Atlanta Stadium                 10
Parc Jarry                       8
Pro Player Stadium               8
American League Park II          7
D.C. Stadium                     7
Cubs Park                        7
Robert F. Kennedy Stadium        6
Globe Life Park in Arlington     6
O.co Coliseum                    5
Joe Robbie Stadium               4
McAfee Coliseum                  4
Ameriquest Field                 3
American League Park

In [28]:
pd.set_option('max.rows', 450)
problem_stadiums

,team_code,team,state_code,year,primary_stadium,primary_latitude,primary_longitude,secondary_stadium,secondary_latitude,secondary_longitude,attendance/game,pitching_park_factor,batting_park_factor,stadium_name,park_id
26,OAK,Oakland Athletics,CA,2015,O.co Coliseum,37.751667,122.200556,NaN,NaN,NaN,"21,829",97,97,NaN,NaN
27,OAK,Oakland Athletics,CA,2014,O.co Coliseum,37.751667,122.200556,NaN,NaN,NaN,"24,736",97,98,NaN,NaN
28,OAK,Oakland Athletics,CA,2013,O.co Coliseum,37.751667,122.200556,NaN,NaN,NaN,"22,337",97,98,NaN,NaN
29,OAK,Oakland Athletics,CA,2012,O.co Coliseum,37.751667,122.200556,NaN,NaN,NaN,"20,729",95,95,NaN,NaN
30,OAK,Oakland Athletics,CA,2011,O.co Coliseum,37.751667,122.200556,NaN,NaN,NaN,"18,232",98,98,NaN,NaN
33,OAK,Oakland Athletics,CA,2008,McAfee Coliseum,37.751667,122.200556,Tokyo Dome,35.705555555555556,139.75194444444443,"20,559",94,94,NaN,NaN
34,OAK,Oakland Athletics,CA,2007,McAfee Coliseum,37.751667,122.200556,NaN,NaN,NaN,"23,726",94,94,NaN,NaN
35,OAK,Oakland Athletics,CA,2006,McAfee Coliseum,37.751667,122.200556,NaN,NaN,NaN,"24,403",97,97,NaN,NaN
36,OAK,Oakland Athletics,CA,2005,McAfee Coliseum,37.751667,122.200556,NaN,NaN,NaN,"26,038",100,100,NaN,NaN
149,SFG,San Francisco Giants,CA,2019,Oracle Park,37.778611,122.389167,NaN,NaN,NaN,"33,429",95,94,NaN,NaN


In [34]:
for k in range(26, 31):
    full_frame.at[k, 'park_id'] = 'OAK01'
for k in range(33, 37):
    full_frame.at[k, 'park_id'] = 'OAK01'
for k in range(149, 165):
    full_frame.at[k, 'park_id'] = 'SFO03'
for k in range(362, 369):
    full_frame.at[k, 'park_id'] = 'CHI11'
full_frame.at[389, 'park_id'] = 'SEA03'
for k in range(442, 459):
    full_frame.at[k, 'park_id'] = 'MIA01'
for k in range(459, 488):
    full_frame.at[k, 'park_id'] = 'CHI12'
for k in range(595, 607):
    full_frame.at[k, 'park_id'] = 'WAS11'
for k in range(607, 610):
    full_frame.at[k, 'park_id'] = 'WAS10'
for k in range(610, 638):
    full_frame.at[k, 'park_id'] = 'MON02'
for k in range(638, 646):
    full_frame.at[k, 'park_id'] = 'MON01'
full_frame.at[730, 'park_id'] = 'NYC15'
for k in range(908, 939):
    full_frame.at[k, 'park_id'] = 'TOR02'
for k in range(939, 951):
    full_frame.at[k, 'park_id'] = 'TOR01'
for k in range(1129, 1179):
    full_frame.at[k, 'park_id'] = 'WAS09'
for k in range(1179, 1189):
    full_frame.at[k, 'park_id'] = 'WAS08'
for k in range(1298, 1315):
    full_frame.at[k, 'park_id'] = 'CIN09'
for k in range(1826, 1829):
    full_frame.at[k, 'park_id'] = 'ATL03'
for k in range(1870, 1880):
    full_frame.at[k, 'park_id'] = 'ATL01'
for k in range(2005, 2028):
    full_frame.at[k, 'park_id'] = 'DET04'
for k in range(2065, 2076):
    full_frame.at[k, 'park_id'] = 'NYC21'
full_frame.at[2108, 'park_id'] = 'NYC21'
for k in range(2204, 2236):
    full_frame.at[k, 'park_id'] = 'ANA01'
for k in range(2241, 2247):
    full_frame.at[k, 'park_id'] = 'ARL02'
for k in range(2253, 2256):
    full_frame.at[k, 'park_id'] = 'ARL02'
for k in range(2289, 2299):
    full_frame.at[k, 'park_id'] = 'WAS10'
full_frame.at[2299, 'park_id'] = 'WAS09'
for k in range(2300, 2316):
    full_frame.at[k, 'park_id'] = 'SAN02'
for k in range(2323, 2351):
    full_frame.at[k, 'park_id'] = 'SAN01'

In [77]:
full_frame.to_csv('retrosheet_br_park_codes_final.csv')
full_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2532 entries, 0 to 2589
Data columns (total 15 columns):
team_code               2532 non-null object
team                    2532 non-null object
state_code              2532 non-null object
year                    2532 non-null int64
primary_stadium         2532 non-null object
primary_latitude        2532 non-null float64
primary_longitude       2532 non-null float64
secondary_stadium       67 non-null object
secondary_latitude      67 non-null object
secondary_longitude     67 non-null object
attendance/game         2532 non-null object
pitching_park_factor    2532 non-null int64
batting_park_factor     2532 non-null int64
stadium_name            2137 non-null object
park_id                 2532 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 316.5+ KB


In [67]:
duplicates = full_frame.groupby('team_code').year.value_counts()

In [68]:
duplicates = duplicates[duplicates > 1]

In [69]:
duplicates

team_code  year
CHW        1910    2
           1911    2
           1912    2
           1962    2
           1963    2
           1964    2
           1965    2
           1966    2
           1967    2
           1968    2
           1969    2
           1970    2
           1971    2
           1972    2
           1973    2
           1974    2
           1975    2
CIN        1900    2
           1901    2
CLE        1901    2
           1902    2
           1903    2
           1904    2
           1905    2
           1906    2
           1907    2
           1908    2
           1909    2
           1910    2
           1911    2
           1912    2
           1913    2
           1914    2
           1915    2
           1928    2
           1929    2
           1930    2
           1931    2
           1934    2
           1935    2
           1936    2
           1937    2
           1938    2
           1939    2
           1940    2
           1941    2
           1942   

In [75]:
full_frame = full_frame.drop_duplicates(['team_code', 'year'])

In [76]:
len(full_frame)

2532

In [2]:
full_frame = pd.read_csv('data/retrosheet_br_park_codes_final.csv', index_col = [0])

In [3]:
full_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2532 entries, 0 to 2589
Data columns (total 15 columns):
team_code               2532 non-null object
team                    2532 non-null object
state_code              2532 non-null object
year                    2532 non-null int64
primary_stadium         2532 non-null object
primary_latitude        2532 non-null float64
primary_longitude       2532 non-null float64
secondary_stadium       67 non-null object
secondary_latitude      67 non-null object
secondary_longitude     67 non-null object
attendance/game         2532 non-null object
pitching_park_factor    2532 non-null int64
batting_park_factor     2532 non-null int64
stadium_name            2137 non-null object
park_id                 2532 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 316.5+ KB


In [4]:
for_merge = full_frame[['team_code', 'year', 'park_id']]

In [7]:
mlb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198286 entries, 0 to 198285
Data columns (total 66 columns):
date                    198286 non-null object
season                  198286 non-null int64
neutral                 198286 non-null int64
playoff                 1608 non-null object
team1                   198286 non-null object
team2                   198286 non-null object
elo1_pre                198286 non-null float64
elo2_pre                198286 non-null float64
elo_prob1               198286 non-null float64
elo_prob2               198286 non-null float64
elo1_post               198286 non-null float64
elo2_post               198286 non-null float64
rating1_pre             198286 non-null float64
rating2_pre             198286 non-null float64
pitcher1                198278 non-null object
pitcher2                198278 non-null object
pitcher1_rgs            184642 non-null float64
pitcher2_rgs            184642 non-null float64
pitcher1_adj            183965 non-nu

In [6]:
mlb = pd.read_csv('data/mlb_final_retro.csv', low_memory = False, index_col = [0])

In [8]:
mlb = mlb.merge(for_merge, how = 'left', left_on = ['year', 'team_code'], right_on = ['year', 'team_code'])

In [9]:
mlb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198286 entries, 0 to 198285
Data columns (total 67 columns):
date                    198286 non-null object
season                  198286 non-null int64
neutral                 198286 non-null int64
playoff                 1608 non-null object
team1                   198286 non-null object
team2                   198286 non-null object
elo1_pre                198286 non-null float64
elo2_pre                198286 non-null float64
elo_prob1               198286 non-null float64
elo_prob2               198286 non-null float64
elo1_post               198286 non-null float64
elo2_post               198286 non-null float64
rating1_pre             198286 non-null float64
rating2_pre             198286 non-null float64
pitcher1                198278 non-null object
pitcher2                198278 non-null object
pitcher1_rgs            184642 non-null float64
pitcher2_rgs            184642 non-null float64
pitcher1_adj            183965 non-nu

In [10]:
issues = mlb[mlb.park_id_x != mlb.park_id_y]
non_missing = issues[issues.park_id_x.notnull()]

In [12]:
non_missing.team_code.value_counts()

CLE    2842
NYY    2673
OAK     593
ATL     110
SFG      73
PIT      67
BOS      60
PHI      57
CHW      48
WSN      43
STL      39
SEA      39
CIN      37
TOR      30
DET      28
LAD      15
TBD      11
SDP      10
FLA       8
HOU       5
ARI       2
MIN       2
CHC       1
NYM       1
Name: team_code, dtype: int64

In [13]:
cle_probs = non_missing[non_missing.team_code == 'CLE']
print(cle_probs.park_id_x.value_counts())
print(cle_probs.park_id_y.value_counts())

CLE06    1832
CLE05     671
CLE07     328
MIL06       3
CAN01       3
COL03       2
FOR03       2
DAY01       1
Name: park_id_x, dtype: int64
CIN05    1714
CIN04     679
TOL01     365
CLE07      81
CLE08       3
Name: park_id_y, dtype: int64


In [34]:
cle_index = list(cle_probs[(cle_probs.park_id_x == 'CLE06') | (cle_probs.park_id_x == 'CLE05')\
                          | (cle_probs.park_id_x == 'CLE07')].index)
for j in cle_index:
    mlb.at[j, 'park_id_y'] = mlb.iloc[j]['park_id_x']

In [15]:
oak_probs = non_missing[non_missing.team_code == 'OAK']
print(oak_probs.park_id_x.value_counts())
print(oak_probs.park_id_y.value_counts())

PHI10    583
LAS01      6
TOK01      4
Name: park_id_x, dtype: int64
ALT01    583
OAK01     10
Name: park_id_y, dtype: int64


In [19]:
oak_index = list(oak_probs[oak_probs.park_id_x == 'PHI10'].index)

In [22]:
for j in oak_index:
    mlb.at[j, 'park_id_y'] = 'PHI10'

In [23]:
atl_probs = non_missing[non_missing.team_code == 'ATL']
print(atl_probs.park_id_x.value_counts())
print(atl_probs.park_id_y.value_counts())

BOS07    58
BOS05    50
FTB01     1
WAR01     1
Name: park_id_x, dtype: int64
BOS08    54
BOS07    50
BOS05     5
ATL02     1
Name: park_id_y, dtype: int64


In [29]:
atl_index = list(atl_probs[(atl_probs.park_id_x == 'BOS07') | (atl_probs.park_id_x == 'BOS05')].index)
for j in atl_index:
    mlb.at[j, 'park_id_y'] = mlb.iloc[j]['park_id_x']

In [35]:
nyy_probs = non_missing[non_missing.team_code == 'NYY']
print(nyy_probs.park_id_x.value_counts())
print(nyy_probs.park_id_y.value_counts())

NYC16    2598
NYC13      73
NYC17       1
NWK01       1
Name: park_id_x, dtype: int64
NYC21    2599
NYC14      73
NYC13       1
Name: park_id_y, dtype: int64


In [36]:
nyy_index = list(nyy_probs[(nyy_probs.park_id_x == 'NYC16')].index)
for j in nyy_index:
    mlb.at[j, 'park_id_y'] = mlb.iloc[j]['park_id_x']

In [37]:
sfg_probs = non_missing[non_missing.team_code == 'SFG']
print(sfg_probs.park_id_x.value_counts())
print(sfg_probs.park_id_y.value_counts())

NYC14    45
NYC13    28
Name: park_id_x, dtype: int64
NYC10    73
Name: park_id_y, dtype: int64


In [38]:
sfg_index = list(sfg_probs[(sfg_probs.park_id_x == 'NYC14') | (sfg_probs.park_id_x == 'NYC13')].index)
for j in sfg_index:
    mlb.at[j, 'park_id_y'] = mlb.iloc[j]['park_id_x']

In [39]:
issues = mlb[mlb.park_id_x != mlb.park_id_y]
non_missing = issues[issues.park_id_x.notnull()]
non_missing.team_code.value_counts()

NYY    75
PIT    67
BOS    60
PHI    57
CHW    48
WSN    43
STL    39
SEA    39
CIN    37
TOR    30
DET    28
LAD    15
CLE    11
TBD    11
OAK    10
SDP    10
FLA     8
HOU     5
ARI     2
ATL     2
MIN     2
NYM     1
CHC     1
Name: team_code, dtype: int64

In [40]:
len(non_missing)

601

In [41]:
non_missing_indices = list(non_missing.index)

In [43]:
mlb = mlb.drop(non_missing_indices, axis= 0)

In [47]:
missing_park_ids = mlb[mlb.park_id_x.isnull()].index

In [49]:
for j in missing_park_ids:
    mlb.at[j, 'park_id_x'] = mlb.iloc[j]['park_id_y']

IndexError: single positional indexer is out-of-bounds

In [50]:
missing_park_ids = mlb[mlb.park_id_x.isnull()].index

In [52]:
mlb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197685 entries, 0 to 198285
Data columns (total 67 columns):
date                    197685 non-null object
season                  197685 non-null int64
neutral                 197685 non-null int64
playoff                 1608 non-null object
team1                   197685 non-null object
team2                   197685 non-null object
elo1_pre                197685 non-null float64
elo2_pre                197685 non-null float64
elo_prob1               197685 non-null float64
elo_prob2               197685 non-null float64
elo1_post               197685 non-null float64
elo2_post               197685 non-null float64
rating1_pre             197685 non-null float64
rating2_pre             197685 non-null float64
pitcher1                197677 non-null object
pitcher2                197677 non-null object
pitcher1_rgs            184221 non-null float64
pitcher2_rgs            184221 non-null float64
pitcher1_adj            183544 non-nu

In [53]:
mlb.to_csv('mlb_final_pre_eda.csv')